In [67]:
import json
import datetime
import time
import pandas as pd
import plotly as py
import cufflinks as cf
cf.go_offline(connected=True)

from requests_html import HTMLSession
session = HTMLSession()

In [63]:
def str_to_timestamp(date):
    obj_datetime = datetime.datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
    return datetime_to_timestamp(obj_datetime)

def datetime_to_timestamp(dt):
    return int(time.mktime(dt.timetuple()))

def transform_ohlc(data):
    result = {}
    if 't' in data.keys():
        result['Date'] = data.pop('t')
        for i, v in enumerate(result['Date']):
            result['Date'][i] = datetime.datetime.fromtimestamp(v)
    if 'o' in data.keys():
        result['Open'] = data.pop('o')
    if 'h' in data.keys():
        result['High'] = data.pop('h')
    if 'l' in data.keys():
        result['Low'] = data.pop('l')
    if 'c' in data.keys():
        result['Close'] = data.pop('c')
    if 'v' in data.keys():
        result['Volume'] = data.pop('v')
        
    df = pd.DataFrame.from_dict(result)
    return df.set_index('Date')
        

def history(symbol, resolution, from_date, to_date=None):
    from_timestamp = str_to_timestamp(from_date + ' 00:00:00' if len(from_date) == 10 else from_date)
    if not to_date:
        to_timestamp = datetime_to_timestamp(datetime.datetime.now())
    else:     
        to_timestamp = str_to_timestamp(to_date + ' 23:59:59' if len(to_date) == 10 else to_date)

    host = 'https://tvc4.forexpros.com/bf9b0cf35ce6ca91b1bba8dd1d70c0c2/1601606035/18/18/88/'
    uri = f'history?symbol={symbol}&resolution={resolution}&from={from_timestamp}&to={to_timestamp}'
    r = session.get(host + uri)
    data = json.loads(r.html.text)
    return transform_ohlc(data)

In [65]:
df = history(6408, 'D', '2020-01-01')

In [70]:
df['Close'].iplot(kind='line')